# 5. Data Augmentation

- [ ] Run at least one experiment with an augmented dataset that includes additional data selected based on training data perplexity under a language model trained the task you’ve selected.

Possible Sources: 

- [ ] the training sets of the other tasks
- [ ] the Broad Twitter Dataset (https://github.com/GateNLP/broad_twitter_corpus),
- [ ] any other dataset we can think of (any creative ideas?)
- [ ] Trump's offensive tweets?

In [ ]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score,classification_report
import sys
sys.path.insert(0, '/work/twitter-nlp')
from boilerplate import load_and_tokenize, pickling, open_jar, func_regex
import numpy as np

In [ ]:
train_txt_emoji = open_jar('/work/twitter-nlp/data/pickle/tweeteval/emoji/pickled_train_text.pkl')
train_labels_emoji = pd.DataFrame(open_jar('/work/twitter-nlp/data/pickle/tweeteval/emoji/pickled_train_labels.pkl'))
val_txt_emoji = open_jar('/work/twitter-nlp/data/pickle/tweeteval/emoji/pickled_val_text.pkl')
val_labels_emoji = pd.DataFrame(open_jar('/work/twitter-nlp/data/pickle/tweeteval/emoji/pickled_val_labels.pkl'))
X_train_txt_emoji = train_txt_emoji.copy()
X_train_txt_emoji.extend(val_txt_emoji)
y_train_emoji = pd.concat([train_labels_emoji,val_labels_emoji], ignore_index=True).astype(int)
emoji_classes = pd.read_csv("/work/twitter-nlp/data/tweeteval/datasets/emoji/mapping.txt", sep = "	", header=None)

In [ ]:
print(len(X_train_txt_emoji), len(list(y_train_emoji[0])))

50000 50000


In [ ]:
merged = pd.DataFrame(zip(X_train_txt_emoji, list(y_train_emoji[0])))

In [ ]:
merged[merged[1] == 18]

,0,1
36,"[In, the, zone, |, @user, #colorsworldwide, #R...",18
59,"[Getting, up, that, ice, wall, one, ax, at, a,...",18
96,"[Caught, by, the, lens, of, Yc, Wong, photo, b...",18
108,"[Photoshooting, with, the, babe, @user, #sunda...",18
122,"[We, had, a, lot, of, office, volunteers, for,...",18
...,...,...
49734,"[#EmptyStates, Day, 3, @user, @user, @, San, F...",18
49740,"[instant, human, just, add, coffee, lavenderch...",18
49766,"[#Day2, #, #TouristMode, #Views, @, The, Coit,...",18
49909,"[ONLINE, Americana, Hat, in, camo, or, grey, D...",18


## Labeling Trumps Twitter Insults
We took trumps insults (provided by the New York Times) and combined those with all his other tweets. 

In [ ]:
# loading in trump's twitter insults
# the insult data is a collection of all insults from 2015 to 2021 (until trump was banned)
insult_data = pd.read_csv("/work/twitter-nlp/data/trumptwitterinsults/trump_insult_tweets_2014_to_2021.csv")

# only using the tokens data, and dropping all duplicates, adding a hatespeech label (all are marked as hatespeech)
tweets = insult_data["tweet"]
tweets = tweets.drop_duplicates()
insult_tokens = pd.DataFrame(columns=["Labels", "Tweets"])
insult_tokens["Tweets"] = tweets
insult_tokens["Labels"] = [1 for _ in range(0, len(insult_tokens))]

In [ ]:
# the standard data is a collection of all of trumps tweets from 2009 to 2021 (until trump was banned)
standard_data = pd.read_csv("/work/twitter-nlp/data/trumptwitterinsults/tweets.csv")

# shortening the data from 2015 to 2009 only, using only tweets, dropping duplcates, adding label
standard_data = standard_data[39087:-1]
k_tweets = standard_data["text"]
k_tweets = k_tweets.drop_duplicates()
control_tokens = pd.DataFrame(columns=["Labels", "Tweets"])
control_tokens["Tweets"] = k_tweets
control_tokens["Labels"] = [0 for _ in range(0, len(control_tokens))]

In [ ]:
# combining the two df
combined_df = pd.concat([insult_tokens, control_tokens])
text = list(combined_df["Tweets"])
combined_df

,Labels,Tweets
0,1,"Can you believe this fool, Dr. Thomas Frieden ..."
2,1,Big time in U.S. today - MAKE AMERICA GREAT AG...
3,1,Politician @SenatorCardin didn't like that I s...
4,1,"For the nonbeliever, here is a photo of @Neily..."
5,1,".@Neilyoung’s song, “Rockin’ In The Free World..."
...,...,...
56565,0,RT @heatherjones333: 'We Have Taken Appropriat...
56566,0,RT @RandPaul: I don’t know why @JoeBiden think...
56567,0,RT @EliseStefanik: President @realDonaldTrump ...
56568,0,RT @TeamTrump: LIVE: Presidential Debate #Deba...


### Predicting based on our previously trained model:

In [ ]:
def classify_sentence(text):
    classifier = open_jar('/work/twitter-nlp/data/pickle/models/hatespeech_model_MultinomialNB.sav')
    cv = open_jar('/work/twitter-nlp/data/pickle/models/hate/vectorizer.pkl')
    return classifier.predict_proba(cv.transform(text).toarray())

In [ ]:
[str(combined_df["Tweets"][0])]

['Can you believe this fool, Dr. Thomas Frieden of CDC, just stated, "anyone with fever should be asked if they have been in West Africa" DOPE']

In [ ]:
predict_list = []

for i in combined_df["Tweets"][0:100]:
    line_data = classify_sentence([str(i)])
    predict_list.append(line_data.tolist())

predict_list

[[[0.943201918603514, 0.056798081396488655]],
 [[0.09583791432069015, 0.9041620856793162]],
 [[0.8647822810031661, 0.13521771899683838]],
 [[0.999632340792054, 0.00036765920795125636]],
 [[0.9955275156159974, 0.004472484384002976]],
 [[0.8727209557419747, 0.1272790442580165]],
 [[0.7818805785544442, 0.2181194214455524]],
 [[0.09410591495534859, 0.9058940850446472]],
 [[0.9649049721566286, 0.03509502784338398]],
 [[0.8199098371940978, 0.1800901628058909]],
 [[0.8371694835282447, 0.16283051647175684]],
 [[0.9707822279353352, 0.02921777206465968]],
 [[0.6846809279341322, 0.31531907206586957]],
 [[0.8581667698699806, 0.14183323013001747]],
 [[0.9962900108796845, 0.0037099891203276505]],
 [[0.050910534095024364, 0.949089465904978]],
 [[0.20894228269711276, 0.7910577173028831]],
 [[0.0494898109888614, 0.950510189011137]],
 [[0.4019431636804088, 0.5980568363195855]],
 [[0.7552670544524318, 0.24473294554756497]],
 [[0.9924835063386216, 0.007516493661378405]],
 [[0.42500715861182065, 0.57499284

In [ ]:
classify_sentence(['hey you useless piece of shit'])

array([[0.40414146, 0.59585854]])

In [ ]:
pred = classifier.predict(X)


ValueError: X has 4 features, but DecisionTreeClassifier is expecting 5000 features as input.

In [ ]:
combined_df["Predicted hs"] = list(pred)
combined_df

NameError: name 'pred' is not defined

In [ ]:
acc_score = accuracy_score(y, pred)

acc_score

NameError: name 'y' is not defined

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=bf1f20fb-a6c0-4ea2-8567-a5a41eea1d8a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>